<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Experience-Bot: Quickly find your demos of interest by just typing
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Install required libraries</b></p>

In [ ]:
%%capture

!pip install openai langchain langchain-openai langchain-community requests faiss-cpu panel==1.3.4

<div class="alert alert-block alert-info">
    <p style='font-size:16px;font-family:Arial;color:#00233C'><b>Note:</b> Please restart the kernel. The simplest way is by typing <b>0 0</b> (zero zero) and then pressing <i><b>enter</b></i></p>
</div>

In [ ]:
import os
import json
import re

# genAI
import openai
from langchain.document_loaders import DirectoryLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

from langchain_community.document_loaders import NotebookLoader
from langchain_community.document_loaders import DirectoryLoader

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


# display
from IPython.display import display, Markdown, HTML, Javascript

#teradataml
from teradataml import *

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO= ExperienceBot.ipynb;' UPDATE FOR SESSION;''')

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Set OpenAI key to environment</b></p>

In [ ]:
import getpass
import os

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Create the embeddings or load from existing</b></p>

In [ ]:
from chat_helper_db import *

In [ ]:
def load_emb():
    # Load the FAISS index with dangerous deserialization enabled
    # Create vector store using embeddings
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large") 

    return FAISS.load_local(
        "notebooks_index", embeddings, allow_dangerous_deserialization=True
    )

<div class="alert alert-block alert-warning">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: You do not have to run the next cell multiple times.  Each time it is executed it will generate over 1M embeddings and the charge is typically $0.02USD / 1M tokens</b>.</p></div>
    
<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Note: If you have previously run this notebook, please select <b>No</b> in response to the following question.</p></div>

In [ ]:
# Request user's input
generate = input("Do you want to generate embeddings? ('yes'/'no'): ")

# Check the user's input
if generate.lower() == "yes":
    vector_store = generate_emb()
elif generate.lower() == "no":
    try:
        print('Loading existing embeddings...')
        vector_store = load_emb()
        print('Embeddings are loaded now...')
    except:
        print('Embeddings not found, generating now..')
        generate_emb()
        vector_store = load_emb()

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Define a RAG Pipeline</b></p>

In [ ]:
# Custom Prompt Template
CUSTOM_PROMPT = """
You are a helpful assistant. Use the following retrieved information from Jupyter notebooks to provide:
1. A **clean and concise textual explanation** based on the question and notebook markdown.
2. **Relevant Clean Python code** extracted from the notebooks' code cells that are related to the question. Please filter the code that is related to the query.
3. Extract the source documents.
If no relevant information is found, politely say so.

*Critical*: start by greeting only if user starts with greeting, just say, "Hey there! 😊 Welcome to our chatbot!"

Context:
{context}

Question:
{question}

Your response should be in below format:
##Answer:
##Relevant Code:
##Source documents:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=CUSTOM_PROMPT
)

# Make sure to use a Chat model like 'gpt-4' or 'gpt-3.5-turbo'
chat_model = ChatOpenAI(model="gpt-4o-mini")

# Retrieval QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    retriever=vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 10}),  # Assuming vector_store is your vector database
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
import panel as pn
pn.extension(design="material")

# panel callback function
def callback(contents, user, instance):
    response = qa_chain.invoke(contents)
    result = response['result']
    
    source_docs = response.get("source_documents", [])
    sources = "\n".join(set([doc.metadata.get("source", "Unknown") for doc in source_docs]))
    html_output = extract_answer_code_references(sources)
    result = result + "\n\n" +  html_output
    return result


pn.chat.ChatInterface(
    callback=callback,
    show_rerun=False,
    show_undo=False,
    show_clear=False,
    width=1200,
    height=400,
)

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>To ensure that the Chatbot interface reflects the latest changes, please reload the page by clicking the 'Reload' button or pressing F5 on your keyboard for <b>first-time only</b> This will update the notebook with the latest modifications, and you'll be able to interact with the Chatbot using the new libraries.</i></p>
    </div>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. You can try your own question</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here are some sample questions that you can try out:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>How VectorDistance works?</li>
    <li>What is Script table operator?</li>
    <li>Give me demos which have AWS Bedrock?</li>
    <li>What is GEOSEQUENCE? Show me some examples</li>
    <li>Which notebooks are using OpenAI?</li>
    <li>Which notebooks are about fraud detection?</li>
    <li>How to use TDApiClient to generate the embeddings?</li>
    <li>Show me demo for Broken digital Journey?</li>
</ol>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>